In [2]:
pip install seaborn


You should consider upgrading via the '/Users/johnrivera/.virtualenvs/Post_Subreddits-R7WzUq5X/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install basilica


You should consider upgrading via the '/Users/johnrivera/.virtualenvs/Post_Subreddits-R7WzUq5X/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn 

You should consider upgrading via the '/Users/johnrivera/.virtualenvs/Post_Subreddits-R7WzUq5X/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install Flask

  Using cached Flask-1.1.2-py2.py3-none-any.whl (94 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached itsdangerous-1.1.0-py2.py3-none-any.whl (16 kB)
  Using cached Werkzeug-1.0.1-py2.py3-none-any.whl (298 kB)
You should consider upgrading via the '/Users/johnrivera/.virtualenvs/Post_Subreddits-R7WzUq5X/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install SQLAlchemy 


You should consider upgrading via the '/Users/johnrivera/.virtualenvs/Post_Subreddits-R7WzUq5X/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import sklearn
import basilica
import decouple
import numpy as np
import sqlite3


In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

In [5]:
df = pd.read_csv("rspct.tsv", sep='\t')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013000 entries, 0 to 1012999
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   id         1013000 non-null  object
 1   subreddit  1013000 non-null  object
 2   title      1013000 non-null  object
 3   selftext   1013000 non-null  object
dtypes: object(4)
memory usage: 30.9+ MB


# Seperate Data Frame into Training and Testing Set

In [7]:
from sklearn.model_selection import train_test_split
features = ['id','selftext']
target = ['subreddit']
X = df[features]
y = df[target]

 

In [8]:
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=.20, random_state=42)

In [9]:
len(X_train)

810400

In [10]:
#want to predict what subreddit a randomtext is more likeley to belong to
len(y_train)

810400

In [11]:
#reduce number of rows to 100; truncation
X_new = X_train.copy()

In [12]:
X_new = X_new.iloc[:10000]
X_new.head()

,id,selftext
654626,60dwoc,I would love to hear your experience with the ...
630792,7bvasp,"Hi all,<lb><lb>Looking to see if anyone recogn..."
934315,56tts7,Hello. I'm a posdoc in the computer science de...
509697,5fv11j,"Hey Guys, <lb><lb>So currently in a situation ..."
451723,7op7n3,I feel like a crazy person with thoughts pouri...


In [13]:
y_train_new= y_train[:10000]
y_train_new

,subreddit
654626,slp
630792,whatsthisplant
934315,bioinformatics
509697,sharepoint
451723,Stellar
...,...
259306,7daystodie
550278,secretsanta
234700,Libraries
829103,FoWtcg


In [14]:
len(y_train_new)

10000

In [15]:
len(X_new)

10000

# Make Embeddings from SQL Lite

In [219]:
BASILICA_KEY ="92d11023-f16d-00d3-92c6-4fd80a32fdc3"
BASILICA = basilica.Connection(BASILICA_KEY)

In [21]:
#embed all the posts into basillica embeddings
all_post_embeddings = []
for text in df['selftext'].iloc[:10]:
    embedding = BASILICA.embed_sentence(text, model='reddit')
    all_post_embeddings.append([np.array(embedding)])


In [341]:
#all_post_embeddings

In [23]:
# I want to get the embeddings from my sqllite data base as appose to the data frame
# Also i want to get the selftext from the sql data base

In [18]:
# make a connection with data base to acces data
conn = sqlite3.connect('Subreddits.db')
c = conn.cursor()
query ='SELECT selftext FROM SUBREDDITS LIMIT 10000;'
#change the limit to 10 for fast execution of code
c.execute(query)
selftext = c.fetchall()


In [19]:
conn = sqlite3.connect('Subreddits.db')
c = conn.cursor()
query2 ='SELECT subreddit FROM SUBREDDITS LIMIT 10000;'
#change the limit to 10 for fast execution of code
c.execute(query2)
subreditts = c.fetchall()


In [20]:
len(subreditts)

10000

In [22]:
len(selftext)

10000

In [30]:
embedded_selftext = []
for x in selftext:
    text = list(x)[0]
    embedding = BASILICA.embed_sentence(text, model='reddit')
    embedded_selftext.append(np.array(embedding))



In [31]:
subreddits_from_data_base = []
for x in subreditts:
    text = list(x)[0]
    subreddits_from_data_base.append(text)


In [24]:
#len(subreddits_from_data_base)

In [25]:
#len(embedded_selftext)

# Making Embeddings from Pandas DataFrame

In [34]:

#create a list with numpy arrays of embeddings
#not sure whether it matters whether the embeddings are in numpy array format or in list
#format for the creation of the model
list1 = []
for text in X_new['selftext']:
    embedding = BASILICA.embed_sentence(text, model='reddit')
    list1.append(np.array(embedding))

In [26]:
#len(list1)


In [27]:
y_train_new

,subreddit
654626,slp
630792,whatsthisplant
934315,bioinformatics
509697,sharepoint
451723,Stellar
...,...
259306,7daystodie
550278,secretsanta
234700,Libraries
829103,FoWtcg


In [28]:
labels2 = [name for name in y_train_new['subreddit']]
#labels2

# Constructing Models

In [37]:
from sklearn.linear_model import LogisticRegression

In [38]:
#Relied from data extracted from pandas data frame
log_reg = LogisticRegression(max_iter=1000).fit(list1, labels2)

In [39]:
#Relied on Data from SQLLITE DATA BASE *
log_reg2 = LogisticRegression(max_iter=1000).fit(embedded_selftext,subreddits_from_data_base,)

# Saving Models Into PickleFormat

In [40]:
import pickle

In [ ]:
filename = open('logistic.dummy','wb')

In [ ]:
pickle.dump(log_reg2,filename)

In [41]:
reg_file = open('logistic.model3','rb')

In [42]:
log_reg = pickle.load(reg_file)

In [56]:
log_reg.predict

<bound method LinearClassifierMixin.predict of LogisticRegression(max_iter=1000)>

In [55]:
reddit_embedding = BASILICA.embed_sentence("I would love to hear your experience with the district as I'm considering applying for the 2017/18 school year. The salary is certainly quite a draw; they seem to pay significantly more than other districts in the area. Is there a particular reason for this? High caseloads, etc.? Any and all information is appreciated. Thanks in advance! ", model='reddit')

In [ ]:
type(reddit_embedding)

# Test Predictions

In [ ]:
prediction1 = log_reg.predict(np.array(reddit_embedding).reshape(1, -1))
prediction1

In [ ]:
prediction2 = log_reg2.predict(np.array(reddit_embedding).reshape(1, -1))
prediction2

In [57]:
log_reg.predict(np.array(reddit_embedding).reshape(1, -1))

array(['surfing'], dtype='<U21')

In [ ]:
prediction1[0]

In [ ]:
prediction2[0]


# Model Accuracy

In [303]:
#Baseline
# Our baseline is .2% for physical therapy
y_train[:10000].value_counts(normalize=True)

subreddit       
physicaltherapy     0.0021
FORTnITE            0.0019
Pauper              0.0019
OCD                 0.0019
cats                0.0019
                     ...  
duelyst             0.0003
uberdrivers         0.0003
thinkpad            0.0003
Eminem              0.0003
TheLastAirbender    0.0002
Length: 1013, dtype: float64

In [ ]:
#compare y_pred and y_train vectors, and compute accuracy of model 3; not model3 used 10,000 reddit post
#10,000 posts represent approx 1% of my data set.

In [220]:
X_new['selftext'].head()

654626    I would love to hear your experience with the ...
630792    Hi all,<lb><lb>Looking to see if anyone recogn...
934315    Hello. I'm a posdoc in the computer science de...
509697    Hey Guys, <lb><lb>So currently in a situation ...
451723    I feel like a crazy person with thoughts pouri...
Name: selftext, dtype: object

In [221]:
log_reg.predict(np.array(reddit_embedding).reshape(1, -1))

array(['surfing'], dtype='<U21')

In [225]:
y_pred = []
for x in X_new['selftext']:
    #print(x)
    embedding = BASILICA.embed_sentence(x, model='reddit')
    prediction = log_reg.predict(np.array(embedding).reshape(1, -1))
    y_pred.append(prediction[0])
    
    

In [226]:
from pandas import DataFrame
y_pred = DataFrame(y_pred,columns=['subreddit'])

In [305]:
X_new['selftext']

654626    I would love to hear your experience with the ...
630792    Hi all,<lb><lb>Looking to see if anyone recogn...
934315    Hello. I'm a posdoc in the computer science de...
509697    Hey Guys, <lb><lb>So currently in a situation ...
451723    I feel like a crazy person with thoughts pouri...
                                ...                        
977204    Hey guys, I usually don’t post things but I’ve...
312735    As a bit of a prank for 1st April, I'd like to...
142606    Wanting to post on here to release some of the...
915786    Hello everyone. I’m a crypto newbie. I’m still...
637017    This is it, reddit. Our chance to do something...
Name: selftext, Length: 1000, dtype: object

In [250]:
#one hot encode data frames
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [240]:
from sklearn.metrics import plot_confusion_matrix, classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
len(y_pred)

10000

In [317]:
y_train_new_labeled = preprocessing.LabelEncoder().fit_transform(y_train_new['subreddit'])

In [320]:
len(y_train_new_labeled)

10000

In [318]:
y_pred_labeled = preprocessing.LabelEncoder().fit_transform(y_pred['subreddit'])

In [319]:
len(y_pred_labeled)

10000

In [322]:
accuracy_score(y_train_new_labeled,y_pred_labeled)*100

0.06999999999999999

In [329]:
one_hot_y_train = OrdinalEncoder().fit_transform(y_train_new['subreddit'].to_numpy().reshape(-1, 1))

In [330]:
len(one_hot_y_train)

10000

In [332]:
one_hot_y_pred = OrdinalEncoder().fit_transform(y_pred['subreddit'].to_numpy().reshape(-1, 1))

In [334]:
len(one_hot_y_pred)

10000

In [336]:
accuracy_score(one_hot_y_train,one_hot_y_pred)*100

0.06999999999999999

In [ ]:
#The model has an accuracy of 7%